# To be run once to setup modules and paths


In [ ]:
#Reload modules without shutting notebook
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pickle
from IPython.display import clear_output

# Do not show warnings
import warnings
warnings.filterwarnings('ignore')

# Global variables including folders
home = os.path.expanduser("~")
notebfolder = os.path.join(home, "notebooks")
basefolder = os.path.join(home, "my_shared_data_folder")
s1slcfolder = os.path.join(basefolder, "s1slc")
datasets = os.path.join(basefolder, "datasets")
baseline_dir = os.path.join(basefolder, "baselines")
                           
# Upland burn modules
import sys
sys.path.append(os.path.join(notebfolder, "utils"))
from baseline_estimate import *

## setup temp folder
tmpfolder = os.path.join(notebfolder, "temp")
if not os.path.exists(tmpfolder):
    os.mkdir(tmpfolder)
tmptiffolder = os.path.join(tmpfolder, "tiffs")
if not os.path.exists(tmptiffolder):
    os.mkdir(tmptiffolder)


# Processing SLC data

In [ ]:
vector_file = os.path.join(tmpfolder, "vector_data_%s.p"%(cstudy))
if os.path.exists(vector_file):
    vector_data = pickle.load(open(vector_file, "rb"))    
    print("Loading {} vectors from {}".format(len(vector_data), vector_file))  
else:    
    print("{} doesn't exist, Creating new vector data".format(vector_file))  
    vector_data = None

print("Getting vector data")
vector_data = get_vector_data(products, vector_data=vector_data)
if os.path.exists(vector_file):
    os.remove(vector_file)
pickle.dump(vector_data, open(vector_file, "wb"))

clear_output(wait=True)

print("Saved {} vectors to {}".format(len(vector_data), vector_file))  


In [ ]:
pairings_baselines = {}
total_done = 0
for p in list_of_potential_pairs:
    file1 = os.path.join(s1slcfolder, p[0])
    file2 = os.path.join(s1slcfolder, p[1])
    total_done +=1
    t1 = vector_data[os.path.basename(file1).strip(".zip")]
    t2 = vector_data[os.path.basename(file2).strip(".zip")]
    dist = closestDistanceBetweenLines(np.array(t1[0]), np.array(t1[-1]), np.array(t2[0]), np.array(t2[-1]))
    pairings_baselines[p[0], p[1]] = dist
    print(p[0], p[1], dist)

In [ ]:
s1slc_analysed_folder = os.path.join(tmpfolder, 'Processed_images/')
if not os.path.exists(s1slc_analysed_folder):
    os.mkdir(s1slc_analysed_folder)
final_folder = os.path.join(basefolder, "%s/coherence_tiffs"%(cstudy))
assert os.path.exists(final_folder), "Final folder %s doesnt exist"%(final_folder)

# Create the baseline files

In [ ]:
file_merge_dict = {}
baseline_dict = {}
                
for p in list_of_potential_pairs:
    file1 = os.path.basename(p[0]).split("_")[-5]
    file2 = os.path.basename(p[1]).split("_")[-5]
    basename = os.path.basename(p[0]).strip(".zip")
    sensor = file1[2] + file2[2]
    for pol in ["VH", "VV"]:
        outfile = file1 + "_" + file2 + "_%s.tif"%(pol)
        orbit_direction = products[products['title'] == basename]['orbitdirection'][0]
        num = products[products['title'] == basename]['relativeorbitnumber'][0]
        merge_id = (num, sensor, outfile[:8], outfile.split("_")[1][:8], orbit_direction, pol)
        if not merge_id in file_merge_dict.keys():
            file_merge_dict[merge_id] = [outfile]
            baseline_dict[merge_id] = [pairings_baselines[p[0], p[1]]]
            continue
        file_merge_dict[merge_id].append(outfile)
        baseline_dict[merge_id].append(pairings_baselines[p[0], p[1]])
            
baseline_file = os.path.join(baseline_dir, "baselines_%s.p"%(cstudy))
if os.path.exists(baseline_file):  
    baseline_perp_dict = pickle.load(open(baseline_file, "rb"))
    print("Loading {} from {}".format(len(baseline_perp_dict), baseline_file))  
else:
    print("Creating new baseline file")  
    baseline_perp_dict = {}
    
for id1, files in file_merge_dict.items():
    outfile = "%s_%s_%s_%s_%s_%s_%s"%(id1[0], id1[1], id1[2], id1[3], id1[4], id1[5], cstudy)
    outfile_tif = os.path.join(final_folder, '%s.tif'%(outfile))
    baseline = np.mean(baseline_dict[id1])
    if outfile_tif in baseline_perp_dict.keys():
        continue
    baseline_perp_dict[outfile_tif] = baseline
    
pickle.dump(baseline_perp_dict, open(baseline_file, "wb"))
                               
#clear_output(wait=True)
print("Saved {} baselines to {}".format(len(baseline_perp_dict), baseline_file))        
